In [1]:
!pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 17.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 30.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 9.9 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.0/6.0 MB 31.4 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 81.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 469.0/469.0 kB 12.8 MB/s eta 0:00:0000:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.5/78.5 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 949.8/949.8 kB 141.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 100.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 21.1 MB/s eta 

In [2]:
import torch
import numpy as np
import random

seed = 0
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
from pathlib import Path

from utils.data_preparation import DataPreparer
from da_fine_tuning.dataset_preparer import DatasetPreparer
from da_fine_tuning.fine_tune_depth_anything import DepthAnythingFineTuner

In [4]:
# Imprtant! Set directory where files will be stored
workspace_path = Path('/workspace/output')
if not workspace_path.exists():
    workspace_path.mkdir()

In [5]:
data_preparer = DataPreparer(workspace_path)
data_preparer.prepare()

test.zip:   0%|          | 0.00/6.78G [00:00<?, ?B/s]

Extracting test dataset


Fix depths in /workspace/output/test_depths: 100%|██████████| 1423/1423 [00:33<00:00, 42.13it/s]


valid.zip:   0%|          | 0.00/6.96G [00:00<?, ?B/s]

Extracting valid dataset


Fix depths in /workspace/output/valid_depths: 100%|██████████| 1441/1441 [00:30<00:00, 46.60it/s]


train.zip:   0%|          | 0.00/19.9G [00:00<?, ?B/s]

Extracting train dataset


Fix depths in /workspace/output/train_depths: 100%|██████████| 3864/3864 [01:25<00:00, 45.09it/s]


In [6]:
dataset_preparer = DatasetPreparer()
dataset_preparer.prepare_dataset(workspace_path / 'train_images', workspace_path / 'train_masks')
dataset_preparer.prepare_dataset(workspace_path / 'valid_images', workspace_path / 'valid_masks')
dataset_preparer.prepare_dataset(workspace_path / 'test_images', workspace_path / 'test_masks')

Prepare masks: 100%|██████████| 1423/1423 [05:30<00:00,  4.30it/s]


In [7]:
da_fine_tuner = DepthAnythingFineTuner(
    depth_train_path=workspace_path / 'train_depths',
    depth_val_path=workspace_path / 'valid_depths',
    depth_test_path=workspace_path / 'test_depths',
)

config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

xFormers not available
xFormers not available


pytorch_model.bin:   0%|          | 0.00/1.34G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/_utils.py:776: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [8]:
da_fine_tuner.adjust_depths_to_pred()

Adjust to pred: 100%|██████████| 3864/3864 [17:15<00:00,  3.73it/s]


In [9]:
da_fine_tuner.fine_tune(n_epochs=24, checkpoint_path=workspace_path)

Epoch 1/24: 100%|██████████| 3864/3864 [30:45<00:00,  2.09it/s, avg_loss=0.148, loss=0.0858] 


Epoch 1/24, Loss: 0.1476


Val error: 100%|██████████| 1441/1441 [06:37<00:00,  3.63it/s, rse=0.855, silog_err=0.418]


Val loss: silog: 0.4183, sq: 0.8547


Epoch 2/24: 100%|██████████| 3864/3864 [30:42<00:00,  2.10it/s, avg_loss=0.0918, loss=0.0411] 


...

Val loss: silog: 0.3050, sq: 0.4899


Epoch 24/24: 100%|██████████| 3864/3864 [30:21<00:00,  2.12it/s, avg_loss=0.0131, loss=0.0397] 


Epoch 24/24, Loss: 0.0131


Val error: 100%|██████████| 1441/1441 [06:24<00:00,  3.75it/s, rse=0.493, silog_err=0.305]

Val loss: silog: 0.3054, sq: 0.4929


In [10]:
# Write predictions

import cv2
from tqdm import tqdm
import os
from pathlib import Path

train_img_path = workspace_path / 'train_images'
train_pred_path = workspace_path / 'train_pred'
train_pred_path.mkdir()
for filename in tqdm(os.listdir(train_img_path), desc=f"Pred train", leave=True):
    depth = da_fine_tuner.predict(train_img_path / filename)
    cv2.imwrite(str(train_pred_path / filename), depth)

valid_img_path = workspace_path / 'valid_images'
valid_pred_path = workspace_path / 'valid_pred'
valid_pred_path.mkdir()
for filename in tqdm(os.listdir(valid_img_path), desc=f"Pred valid", leave=True):
    depth = da_fine_tuner.predict(valid_img_path / filename)
    cv2.imwrite(str(valid_pred_path / filename), depth)

test_img_path = workspace_path / 'test_images'
test_pred_path = workspace_path / 'test_pred'
test_pred_path.mkdir()
for filename in tqdm(os.listdir(test_img_path), desc=f"Pred test", leave=True):
    depth = da_fine_tuner.predict(test_img_path / filename)
    cv2.imwrite(str(test_pred_path / filename), depth)

Pred test: 100%|██████████| 1423/1423 [07:37<00:00,  3.11it/s]


## Fine tune predictions for players

In [11]:
from pathlib import Path

In [12]:
from players_fine_tuning.segment import PlayerSegmenter

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


In [13]:
segmenter = PlayerSegmenter()

100%|██████████| 137M/137M [00:02<00:00, 68.3MB/s] 


In [14]:
train_segments_path = workspace_path / 'train_segments'
val_segments_path = workspace_path / 'valid_segments'
test_segments_path = workspace_path / 'test_segments'

train_segments_path.mkdir(), val_segments_path.mkdir(), test_segments_path.mkdir()

segmenter.segment(workspace_path / 'train_images', train_segments_path)
segmenter.segment(workspace_path / 'valid_images', val_segments_path)
segmenter.segment(workspace_path / 'test_images', test_segments_path)

Process segmentation: 100%|██████████| 1423/1423 [09:16<00:00,  2.56it/s]


In [15]:
from players_fine_tuning.dataset_preparer import DatasetPreparer

player_ds_preparer = DatasetPreparer(
    train_segments_path = Path(train_segments_path),
    val_segments_path = Path(val_segments_path),
    test_segments_path = Path(test_segments_path),

    train_orig_depth_path = workspace_path / 'train_depths',
    val_orig_depth_path = workspace_path / 'valid_depths',
    test_orig_depth_path = workspace_path / 'test_depths',

    train_pred_path = train_pred_path,
    val_pred_path = valid_pred_path,
    test_pred_path = test_pred_path,
)

In [16]:
player_ds_preparer.prepare_dataset()

Process test_segments: 100%|██████████| 1423/1423 [01:50<00:00, 12.89it/s]


In [17]:
from pathlib import Path

from players_fine_tuning.player_fine_tuner import PlayerFineTuner

player_fine_tuner = PlayerFineTuner(workspace_path / 'train_segments', workspace_path / 'valid_segments')

In [18]:
player_fine_tuner.fine_tune(n_epochs=10)

Epoch 1/10: 100%|██████████| 3389/3389 [03:04<00:00, 18.33it/s, avg_loss=0.000139, loss=0.000114]


Epoch 1/10, Loss: 0.0001


Val error: 100%|██████████| 334/334 [00:47<00:00,  7.10it/s, loss=0.000134]


Val loss: 0.00013426, self._best_score=0.00013425593441028098


Epoch 2/10: 100%|██████████| 3389/3389 [03:02<00:00, 18.58it/s, avg_loss=0.000119, loss=8.4e-5]  


...

Val loss: 0.00012109, self._best_score=0.00011935952308791282


Epoch 10/10: 100%|██████████| 3389/3389 [03:04<00:00, 18.36it/s, avg_loss=8.27e-5, loss=9.36e-5] 


Epoch 10/10, Loss: 0.0001


Val error: 100%|██████████| 334/334 [00:50<00:00,  6.61it/s, loss=0.000121]


Val loss: 0.00012139, self._best_score=0.00011935952308791282


In [19]:
player_fine_tuner.save(workspace_path / 'player_ft.pth')

## Apply fine tuned players depths

In [20]:
import os
import numpy as np
import torch
import tempfile
from torch.nn import functional as F

from players_fine_tuning.dataset_loader import PlayerDataset

masks_path = workspace_path / 'test_segments' / 'masks'

results_path = workspace_path / 'results'
results_path.mkdir()

group_by_name = {}
for filename in os.listdir(masks_path):
    name = '_'.join(filename.split('_')[:3])
    if name not in group_by_name:
        group_by_name[name] = []
    group_by_name[name].append(filename)

for group_name, filenames in tqdm(group_by_name.items(), desc=f'Predict', leave=True):

    depth = cv2.imread(str(workspace_path / 'test_pred' / f'{group_name}.png'), cv2.IMREAD_UNCHANGED)
    depth = depth / 255 ** 2
    depth = depth - depth.min()
    depth = depth / depth.max()

    for filename in filenames:
        mask = np.load(masks_path / filename)
        x1, y1, x2, y2 = map(int, filename.replace('.npy', '').split('_')[3:])

        player_depth = PlayerDataset.load_image(workspace_path / 'test_segments' / 'depths' / filename, PlayerDataset.SIZE)
        player_depth = player_depth.unsqueeze(0).unsqueeze(0).to('cuda')
        with torch.no_grad():
            pred = player_fine_tuner.predict(player_depth)

        pred = F.interpolate(pred, (y2-y1, x2-x1), mode='bilinear', align_corners=False)
        
        pred = pred.squeeze(0).squeeze(0).cpu().numpy()
        depth[y1:y2, x1:x2][mask] += pred[mask] / 2

    depth = np.round(depth * 255 ** 2).astype(np.uint16)

    _, game_id, idx = group_name.split('_')
    filename = f'foot_game_{game_id}_video_1_depth_r_{idx}.png'
    cv2.imwrite(str(results_path / filename), depth)

Predict: 100%|██████████| 1423/1423 [04:17<00:00,  5.53it/s]


## Evaluate on test set

In [21]:
from utils.misc import compute_scale_and_shift_np

gt_path = workspace_path / 'test_depths'
pred_path = workspace_path / 'results'
mask_path = workspace_path / 'test_masks'

mse_err, silog_err, cnt = 0.0, 0.0, 0

for filename in tqdm(os.listdir(gt_path), desc=f'Final val', leave=True):
    gt = cv2.imread(gt_path / filename, cv2.IMREAD_UNCHANGED) / 255 ** 2
    _, game_id, idx = filename.replace('.png', '').split('_')
    pred_filename = f'foot_game_{game_id}_video_1_depth_r_{idx}.png'
    pred = cv2.imread(pred_path / pred_filename, cv2.IMREAD_UNCHANGED) / 255 ** 2
    mask = ~np.load(mask_path / filename.replace('.png', '.npy'))
    scale, shift = compute_scale_and_shift_np(pred, gt, mask.astype(np.float32))
    pred = scale * pred + shift
    mse_err += ((pred - gt) ** 2)[mask].mean()
    log_err = np.log(pred) - np.log(gt)
    silog_err += np.sqrt(np.mean(log_err ** 2) - np.mean(log_err) ** 2) * 100

print(f'MSE: {(mse_err / len(os.listdir(gt_path))):.8f}, silog: {(silog_err / len(os.listdir(gt_path))):.8f}')

Final val: 100%|██████████| 1423/1423 [02:10<00:00, 10.90it/s]

MSE: 0.00000301, silog: 0.25562687
